[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/camenduru/latent-consistency-model-colab/blob/main/wip/lcm_controlnet_canny_draw_colab.ipynb)

In [ ]:
!pip install git+https://github.com/huggingface/diffusers -U
!pip install -q controlnet-aux transformers accelerate peft gradio==3.50.2
!pip install -q https://download.pytorch.org/whl/cu118/xformers-0.0.22.post4%2Bcu118-cp310-cp310-manylinux2014_x86_64.whl

In [ ]:
from diffusers import StableDiffusionControlNetPipeline, UNet2DConditionModel, ControlNetModel, LCMScheduler
import torch

controlnet = ControlNetModel.from_pretrained("lllyasviel/control_v11p_sd15_canny", torch_dtype=torch.float16)
pipe = StableDiffusionControlNetPipeline.from_pretrained("Linaqruf/anything-v3.0", controlnet=controlnet, torch_dtype=torch.float16, safety_checker=None,).to("cuda")
pipe.scheduler = LCMScheduler.from_config(pipe.scheduler.config)
pipe.load_lora_weights("latent-consistency/lcm-lora-sdv1-5")
# pipe.fuse_lora()

In [ ]:
from diffusers.utils import load_image, make_image_grid
import cv2
import numpy as np
from PIL import Image
# original_image = load_image("https://image.civitai.com/xG1nkqKTMzGDvpLrqFT7WA/fab28432-7d00-43c4-fbd6-ac5989ebf000/width=1280/326152.jpeg")
input_image = load_image("https://image.civitai.com/xG1nkqKTMzGDvpLrqFT7WA/fab28432-7d00-43c4-fbd6-ac5989ebf000/width=512/326152.jpeg")
input_image = np.array(input_image)
input_image = cv2.Canny(input_image, 100, 200)
input_image = input_image[:, :, None]
input_image = np.concatenate([input_image, input_image, input_image], axis=2)
canny_image = Image.fromarray(input_image)

In [ ]:
image = pipe(f"cute girl", image=canny_image, num_inference_steps=4, guidance_scale=1.0, controlnet_conditioning_scale=0.5, cross_attention_kwargs={"scale": 1.0}).images[0]
make_image_grid([canny_image, image], rows=1, cols=2)